<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/ICG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import torch
from torch import nn
import numpy as np
import pandas as pd
torch.manual_seed(11)
np.random.seed(11)

In [74]:
model = nn.Sequential(
    nn.Linear(14,128),
    nn.ReLU(),
    nn.Linear(128,1))

In [75]:
df = pd.DataFrame(np.random.uniform(size=(1,14)))*100
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,18.026969,1.947524,46.321853,72.493393,42.02036,48.54271,1.278081,48.737161,94.180665,85.079509,72.996447,10.873607,89.390417,85.715425


In [76]:
model(torch.FloatTensor(df.values))

tensor([[5.4128]], grad_fn=<AddmmBackward>)

In [77]:
def ICG(x):
  '''
  # input : only one row pd.DataFrame
  '''
  count = 0
  while True:
    output = model(torch.FloatTensor(x.values))
    if output< 5:
      return output
    else:
      x.iloc[:,-1] -= 30
      count += 1
      print(count,x.iloc[:,-1].values[0])

In [78]:
ICG(df)

1 55.71542470728296


tensor([[3.5570]], grad_fn=<AddmmBackward>)